# Korean vishing Detection using KoBERT model from HuggingFace on TensorFlow

## Install dependencies

Install the tensorFlow to work on GPU for WSL2 Ubuntu 20.04 LTS (Focal Fossa) on Windows 11 Pro as follows described in the [TensorFlow documentation](https://www.tensorflow.org/install/pip#windows-wsl2).
1. Install WSL2
2. Install the NVIDIA drive
3. NVIDIA’s setup docs for CUDA in WSL2
4. Install TensorFlow
5. Verify the installation
6. Install the transformers library
7. Install the pandas library
8. Install the numpy library
9. Install the scikit-learn library
10. Install the matplotlib library
11. Install the seaborn library
(optional)
12. Install the tqdm library
13. Install the sentencepiece library
14. Install the pydot library
15. Install the graphviz library

In [7]:
# Verify the installation:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Check if TensorFlow is using GPU
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Num GPUs Available:  1
Num GPUs Available:  1
Default GPU Device: /device:GPU:0


2023-12-07 19:24:31.485070: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-07 19:24:31.485271: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-07 19:24:31.485315: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-07 19:24:31.492842: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-07 19:24:31.492883: I tensorflow/core/co

In [3]:
!pip install transformers pandas numpy

/bin/bash: /home/mbmk92/anaconda3/envs/dltest/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [5]:
!pip install --upgrade pip

/bin/bash: /home/mbmk92/anaconda3/envs/dltest/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [ ]:
# !pip install tensorflow
# !pip3 install tensorflow[and-cuda]

In [8]:
# load the libraries
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import AdamWeightDecay
import pandas as pd
from sklearn.model_selection import train_test_split